## Trabalhando com o algoritmo de classificação ID3 (Decision Tree)

Como mostralo em aulas, o algoritmo de classificação ID3 necessita de uma regra geral chamada de Ganho de Informação, para indicar quais são os atributos a serem selecionados desde a raíz até um ramo que determina a folha (resposta do problema).

A seguir são apresentados o carregamento dos dados simples do exemplo "Jogar Tênis" e a função que devolve o Ganho de Informação para um conjunto.

In [2]:
from math import log
import numpy as np


def entropia(q):
    if q in (0, 1):
        return 0
    return -(q * log(q, 2) + (1 - q) * log(1 - q, 2))


def ganho_info(ds, base_attr, value=None):
    if value:
        ds = ds[ds[:, base_attr] == value]
        
    classes = set(ds[:, -1])
    a = [len(ds[ds[:, -1] == c]) for c in classes]
    
    
    h = entropia(float(a[0]) / sum(a))
    a_obj = list()
    for value in set(ds[:, base_attr]):
        ds_obj = ds[ds[:, base_attr] == value]
        a_obj.append([len(ds_obj[ds_obj[:, -1] == c]) for c in classes])
    
    sobra = 0
    for d in a_obj:
        print d[0], d[1], a[0], a[1]
        sobra += (float(d[0] + d[1]) / (a[0] + a[1])) * entropia(float(max(d[0], d[1])) / (d[0] + d[1]))
    
    return h - sobra


def main():
    dataset = np.array([
        ['D1', 'sol', 'quente', 'alta', 'fraco', 'não'],
        ['D2', 'sol', 'quente', 'alta', 'forte', 'não'],
        ['D3', 'nublado', 'quente', 'alta', 'fraco', 'sim'],
        ['D4', 'chuva', 'suave', 'alta', 'fraco', 'sim'],
        ['D5', 'chuva', 'frio', 'normal', 'fraco', 'sim'],
        ['D6', 'chuva', 'frio', 'normal', 'forte', 'não'],
        ['D7', 'nublado', 'frio', 'normal', 'forte', 'sim'],
        ['D8', 'sol', 'suave', 'alta', 'fraco', 'não'],
        ['D9', 'sol', 'frio', 'normal', 'fraco', 'sim'],
        ['D10', 'chuva', 'suave', 'normal', 'fraco', 'sim'],
        ['D11', 'sol', 'suave', 'normal', 'forte', 'sim'],
        ['D12', 'nublado', 'suave', 'alta', 'forte', 'sim'],
        ['D13', 'nublado', 'quente', 'normal', 'fraco', 'sim'],
        ['D14', 'chuva', 'suave', 'alta', 'forte', 'não']
    ])
    
    attrs = range(1, len(dataset[0]) - 1)
    print attrs
    ganhos = [ganho_info(dataset, attr) for attr in attrs]
    print ganhos
    print max(ganhos)

main()

[1, 2, 3, 4]
2 3 5 9
3 2 5 9
0 4 5 9
1 3 5 9
2 4 5 9
2 2 5 9
4 3 5 9
1 6 5 9
3 3 5 9
2 6 5 9
[0.24674981977443922, 0.029222565658954758, 0.15183550136234147, 0.04812703040826938]
0.246749819774


No código acima, e mostrado como obter o nó raiz da ID3 utilizando o ganho de informação. Uma boa tarefa para aumentar o conhecimento na manipulação de dados em python e numpy é completar os ramos dessa ID3 utilizando como limiar uma quantidade de classes respondidas pelos testes em cada ramo.

No exemplo abaixo, é mostrado como carregar uma base de dados da UCI e utilizar o classificador ID3 obtido por meio scikit learn. A base de dados é a *Car Evaluation Data Set* ([https://archive.ics.uci.edu/ml/datasets/Car+Evaluation](https://archive.ics.uci.edu/ml/datasets/Car+Evaluation)).

O *dataset* possui os seguintes atributos:
* buying: vhigh, high, med, low. 
* maint: vhigh, high, med, low. 
* doors: 2, 3, 4, 5more. 
* persons: 2, 4, more. 
* lug_boot: small, med, big. 
* safety: low, med, high. 

E as seguintes classes: unacc, acc, good, vgood

### Carregamento da base de dados:

In [3]:
from urllib2 import urlopen

url = 'https://archive.ics.uci.edu/ml/machine-learning-databases/car/car.data'
filedata = urlopen(url)
data = filedata.read()
dataset = np.array([s.split(',') for s in data.split('\n')][:-1])
print dataset
print len(dataset)
print dataset.shape

[['vhigh' 'vhigh' '2' ... 'small' 'low' 'unacc']
 ['vhigh' 'vhigh' '2' ... 'small' 'med' 'unacc']
 ['vhigh' 'vhigh' '2' ... 'small' 'high' 'unacc']
 ...
 ['low' 'low' '5more' ... 'big' 'low' 'unacc']
 ['low' 'low' '5more' ... 'big' 'med' 'good']
 ['low' 'low' '5more' ... 'big' 'high' 'vgood']]
1728
(1728, 7)


O *dataset* foi carregado diretamente da página da internet utilizando `urllib2` e então carregado corretamente em um `numpy.array`.

### Agora é aplicado o processo de aprendizagem utilizando o classificador ID3 do `scikit learn`:

In [4]:
from sklearn import tree
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

# transformação dos valores de categórico para numérico
numerical_features = list()
features = dataset[:, :-1].T
le = LabelEncoder()
for v in features:
    numerical_features.append(le.fit_transform(v))
features = np.array(numerical_features).T
print features
print features.shape
# obtendo a coluna com as respostas
labels = dataset[:, -1]

[[3 3 0 0 2 1]
 [3 3 0 0 2 2]
 [3 3 0 0 2 0]
 ...
 [1 1 3 2 0 1]
 [1 1 3 2 0 2]
 [1 1 3 2 0 0]]
(1728, 6)


In [8]:
X_train, X_test, Y_train, Y_test = train_test_split(features, labels, test_size=0.3, shuffle=True)

clf = tree.DecisionTreeClassifier()

clf.fit(X_train, Y_train)
res = clf.predict(X_test)
misses, hits = 0, 0
for i in range(len(res)):
    if res[i] == Y_test[i]:
        hits += 1
    else:
        misses += 1

print ("acurácia: %.02f%%" % (hits / float(hits + misses) * 100))

acurácia: 96.34%
